In [1]:
%run -p ../train_cnf_disentangle_rl.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 8000 --test_batch_size 5000 --save ../experiments_published/cnf_conditional_disentangle_cifar10_bs8K_sratio_0_5_drop_0_5_rl_stdscale_30_run1 --seed 1 --conditional True --controlled_tol True --train_mode semisup --lr 0.01 --warmup_iters 1000 --atol 1e-4  --rtol 1e-4 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 30.0 --max_grad_norm 20.0
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
GATES = ["cnn1", "cnn2", "rnn"]

parser = argparse.ArgumentParser("Continuous Normalizing F

Files already downloaded and verified
Files already downloaded and verified


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

KeyboardInterrupt: 

         959233208 function calls (943458035 primitive calls) in 59097.669 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      852 35637.215   41.828 35637.215   41.828 {method 'run_backward' of 'torch._C._EngineBase' objects}
   236414 19762.828    0.084 19762.828    0.084 {method 'acquire' of '_thread.lock' objects}
  8236000  352.874    0.000  601.195    0.000 train_cnf_disentangle_rl.py:142(add_noise)
   325143  284.940    0.001  284.940    0.001 {method '_write_file' of 'torch._C.CudaFloatStorageBase' objects}
  8236141  220.330    0.000  220.330    0.000 {method 'tobytes' of 'numpy.ndarray' objects}
  8236200  198.050    0.000  198.050    0.000 {method 'uniform_' of 'torch._C._TensorBase' objects}
  8236000  188.273    0.000  855.335    0.000 functional.py:32(to_tensor)
  8236000  139.554    0.000  139.554    0.000 {method 'div' of 'torch._C._TensorBase' objects}
      141  108.371    0.769  108.371    0.769 {method